<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Time Series Analysis - Foundation Model Chronos-2 - Weather Australia
</b></font> </br></p>

---

**Chronos-2** ist das neueste vortrainierte Zeitreihenmodell von Amazon (veröffentlicht Oktober 2025), das Zeitreihen als Sequenzen von Token wie ein Sprachmodell behandelt. Im Gegensatz zu den Vorgängermodellen unterstützt Chronos-2 **univariate, multivariate und covariate-basierte Prognosen** ohne zusätzliches Training (zero-shot). Das Modell nutzt große Mengen heterogener Zeitreihen zur Vortrainierung und arbeitet vollständig probabilistisch, erzeugt also Vorhersageverteilungen statt nur Punktwerte. Technisch basiert es auf Transformer-Architekturen und discretisiert die Werte in Token, was eine einheitliche Verarbeitung verschiedener Skalen und Domänen ermöglicht.

[Chronos GitHub](https://github.com/amazon-science/chronos-forecasting)   
[Chronos-2 Announcement](https://www.amazon.science/blog/introducing-chronos-2-from-univariate-to-universal-forecasting)    
[AWS Blog: Chronos-Bolt](https://aws.amazon.com/blogs/machine-learning/fast-and-accurate-zero-shot-forecasting-with-chronos-bolt-and-autogluon/)  
[Hugging Face Model](https://huggingface.co/amazon/chronos-2)


[Notebook mit Beispielen: Getting Started with Chronos-2](https://colab.research.google.com/github/amazon-science/chronos-forecasting/blob/main/notebooks/chronos-2-quickstart.ipynb)


# **0 | Install & Import**
---

In [1]:
# Install
!uv pip install chronos-forecasting

Using Python 3.12.12 environment at: /usr
Resolved 54 packages in 793ms
Prepared 5 packages in 1.06s
Installed 5 packages in 256ms
 + boto3==1.40.73
 + botocore==1.40.73
 + chronos-forecasting==2.0.1
 + jmespath==1.0.1
 + s3transfer==0.14.0


In [6]:
# Import
import numpy as np
import pandas as pd
import torch

from chronos import Chronos2Pipeline
from sklearn.metrics import r2_score
from statsmodels.tsa.seasonal import seasonal_decompose

import tensorflow as tf

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# Warnung ausstellen
import warnings
warnings.filterwarnings('ignore')

In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
!nvidia-smi

Fri Nov 14 08:15:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **1 | Understand**
---

<p><font color='black' size="5">
Anwendungsfall
</font></p>







Dieser Datensatz enthält ungefähr 10 Jahre Wetterbeobachtungen von Hobart in Australien.

Die `Mittlere Temperatur` ist die vorherzusagende Zielvariable.
Prognostizieren die `Mittlere Temperatur`, indem Sie das Modelle mit der Zielvariablen trainieren.


[Info](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package)

[DataSet](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package)





In [9]:
# Daten einlesen
df = pd.read_csv('https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/weather_hobart.csv', sep=';')
df['YearMonth'] = pd.to_datetime(df['YearMonth'])
df['MedTemp'] = pd.to_numeric(df['MedTemp'])
df = df.set_index('YearMonth')

In [10]:
# Saisonale Dekomposition - additive Zerlegung
result = seasonal_decompose(df['MedTemp'], model='additive', period=12)

In [11]:
# 4 Subplots erstellen
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'),
    vertical_spacing=0.08
)

# Die vier Komponenten hinzufügen
fig.add_trace(
    go.Scatter(x=result.observed.index, y=result.observed, name='Observed', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=result.trend.index, y=result.trend, name='Trend', line=dict(color='orange')),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=result.seasonal.index, y=result.seasonal, name='Seasonal', line=dict(color='green')),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=result.resid.index, y=result.resid, name='Residual', line=dict(color='red')),
    row=4, col=1
)

# Layout anpassen
fig.update_layout(
    height=1150,
    width=1000,
    showlegend=False,
    title_text="Seasonal Decomposition"
)

# X-Achsen für alle Subplots formatieren
fig.update_xaxes(tickformat='%Y-%m', tickangle=45)

fig.show()

<p><font color='black' size="5">
Analyse der Zeitreihe (durch ChatGPT)
</font></p>

<details>

Die Zerlegung zeigt ein recht typisches Bild für Temperaturdaten, aber ein paar Punkte stechen hervor, wenn man alle vier Komponenten zusammen betrachtet.

---

**1. Observed – Rohdaten**

Die beobachtete Zeitreihe enthält:

* **Starke jährliche Schwankungen**: Hohe Werte im Sommer, tiefe im Winter.
* **Relativ stabile Amplitude** über die Jahre – keine massiven Ausschläge.
* **Leichte Aufwärtstendenz** über den Gesamtzeitraum.

Die Rohkurve wird vor allem durch Saisonalität dominiert, nicht durch langfristige Trends.

---

**2. Trend – langfristige Entwicklung**

Der Trend zeigt:

* **2009–2010**: leichter Anstieg.
* **2010–2011**: spürbarer Rückgang, vermutlich durch ein kaltes Jahr/Winter.
* **2011–2013**: erneuter Anstieg, dann wieder ein deutlicher Einbruch 2013.
* **Ab 2013/2014**: stabiler Aufwärtstrend mit Peak 2016.
* **Gesamtbild**: moderate Erwärmung über die Jahre, aber mit **klaren temporären Abkühlungsphasen**.

Was man nicht unterschätzen sollte:

* Die Trendlinie ist **geglättet** (STL oder ähnlich).
* Kurze Täler (z. B. 2013) können extreme Winter oder Anomalien widerspiegeln, aber nicht zwingend einen echten „Trendwechsel“.

---

**3. Seasonal – wiederkehrendes Muster**

Die saisonale Komponente ist:

* **hochgradig stabil** über alle Jahre hinweg.
* Form und Amplitude ändern sich kaum.
* Die Peaks und Täler liegen fast exakt an den gleichen Kalenderpunkten.

Interpretation:

* Die Temperatur ist stark **jahreszeitabhängig**, und diese Schwankung ist **strukturell konstant**.
* Kein Hinweis auf veränderte Saisonalität (z. B. verschobene Sommer-/Winterpunkte).

Das ist oft ein Zeichen, dass das Klima im betrachteten Zeitraum **nicht saisonal instabil** ist, sondern die langfristigen Änderungen eher im Trend stattfinden.

---

**4. Residual – zufällige Schwankungen**

Die Residuen zeigen:

* Positive und negative Ausschläge, aber **keine systematische Struktur mehr**.
* Einige Jahre mit stärkeren Ausschlägen (z. B. 2010, 2013, 2015).
* Keine erkennbare Autokorrelation mehr – ideal aus Sicht eines Decomposition-Modells.

Bedeutung:

* Das Modell hat die drei dominierenden Muster (Trend, Saison, Rest) sauber getrennt.
* Die Residuals spiegeln Wetteranomalien wider: ungewöhnlich kalte Sommer, extrem warme Frühjahre usw.

---

**5. Gesamtinterpretation**

Wenn man alle Komponenten zusammenführt:

* Die Temperaturen zeigen eine **klare, robuste jährliche Saisonalität**, die konstant bleibt.
* Der **langfristige Trend** weist auf eine leichte Erwärmung hin, unterbrochen von temporären Abkühlungsphasen.
* Die größten Ausschläge im Residualbereich entsprechen wahrscheinlich **Wetterextremen**.
* Die Rohdaten sind stark saisonal geprägt, und der Trend ist im Vergleich zur Saison relativ schwach.

<p><font color='black' size="5">
Additive Zerlegung
</font></p>

<details>

Bei `model='additive'` wird die Zeitreihe in drei Komponenten **addiert**:

```
Y(t) = Trend(t) + Saisonalität(t) + Rest(t)
```

**Die Komponenten:**

*Trend*
- Langfristige Entwicklung (steigend/fallend/konstant)
- Wird durch gleitenden Durchschnitt (Moving Average) ermittelt

*Saisonalität*
- Wiederkehrende Muster mit fester Periode (z.B. 12 Monate)
- Mittelt sich über eine Periode zu **0**
- Konstante Amplitude unabhängig vom Niveau

*Rest (Residual)*
- Unregelmäßige Schwankungen, "Rauschen"
- Was übrig bleibt: `Rest = Original - Trend - Saisonalität`
- Sollte idealerweise zufällig verteilt sein

**Wann additiv?**     
Wenn die saisonalen Schwankungen **konstant** bleiben, egal ob die Werte insgesamt hoch oder niedrig sind. Beispiel: Temperatur schwankt immer um ±5°C zwischen Sommer und Winter, unabhängig vom langfristigen Trend.

<p><font color='black' size="5">
Multiplikative Zerlegung
</font></p>

<details>

Bei `model='multiplicative'` werden die Komponenten **multipliziert**:

```
Y(t) = Trend(t) × Saisonalität(t) × Rest(t)
```

**Die Komponenten:**

*Trend*
- Langfristige Entwicklung (wie bei additiv)
- Hat die gleiche Einheit wie die Originaldaten

*Saisonalität*
- Wiederkehrende prozentuale Schwankungen
- Mittelt sich über eine Periode zu **1.0**
- Werte wie 1.2 = +20% über Trend, 0.8 = -20% unter Trend
- Amplitude wächst/schrumpft **proportional** zum Niveau

*Rest (Residual)*
- Unregelmäßige Schwankungen als Faktor
- Zentriert um 1.0
- Berechnet als: `Rest = Original / (Trend × Saisonalität)`

**Wann multiplikativ?**
Wenn die saisonalen Schwankungen **proportional** zum Niveau sind. Beispiel: Verkaufszahlen schwanken im Dezember immer +30% (nicht +100 Stück), egal ob der Gesamtumsatz bei 1.000 oder 5.000 liegt.

Hinweis:       
Erfordert **positive Werte** (keine Null oder Negativwerte), da durch Division/Multiplikation zergelegt wird.

# **2 | Prepare**
---


In [12]:
# Kontext vorbereiten - Training nur auf Daten bis -12 Monate
# Chronos-2 benötigt einen DataFrame im "Long-Format" mit item_id, timestamp und target
df_train = df[:-12].copy()
prediction_length = 24  # 12 Monate für Vergleich + 12 Monate echte Zukunftsprognose

# Erstelle DataFrame im Format für Chronos-2 mit garantiert regelmäßigen Timestamps
# Chronos-2 kann die Frequenz nur erkennen, wenn die Zeitstempel perfekt regelmäßig sind
start_date = df_train.index[0]
end_date = df_train.index[-1]

# Erstelle perfekt regelmäßigen monatlichen Index
regular_index = pd.date_range(start=start_date, end=end_date, freq='MS')

# Reindex die Daten auf den regelmäßigen Index
df_train_regular = df_train.reindex(regular_index)

# Erstelle DataFrame für Chronos-2
context_df = pd.DataFrame({
    'item_id': 'weather_hobart',
    'timestamp': df_train_regular.index,
    'target': df_train_regular['MedTemp'].values
})

print(f"Trainingsdaten: {len(context_df)} Monate (bis {df.index[-13].strftime('%Y-%m')})")
print(f"Testdaten (letzte 12 Monate): {df.index[-12].strftime('%Y-%m')} bis {df.index[-1].strftime('%Y-%m')}")
print(f"Prognose: {prediction_length} Monate")
print(f"\nContext DataFrame Shape: {context_df.shape}")
print(f"Inferred Frequency: {pd.infer_freq(context_df['timestamp'])}")
print(f"Timestamp dtype: {context_df['timestamp'].dtype}")
print("\nErste Zeilen:")
print(context_df.head())
print("\nLetzte Zeilen:")
print(context_df.tail())

Trainingsdaten: 96 Monate (bis 2016-06)
Testdaten (letzte 12 Monate): 2016-07 bis 2017-06
Prognose: 24 Monate

Context DataFrame Shape: (96, 3)
Inferred Frequency: MS
Timestamp dtype: datetime64[ns]

Erste Zeilen:
          item_id  timestamp     target
0  weather_hobart 2008-07-01   6.000000
1  weather_hobart 2008-08-01   8.135886
2  weather_hobart 2008-09-01   9.686012
3  weather_hobart 2008-10-01  13.867071
4  weather_hobart 2008-11-01  14.884001

Letzte Zeilen:
           item_id  timestamp     target
91  weather_hobart 2016-02-01  19.304568
92  weather_hobart 2016-03-01  18.214254
93  weather_hobart 2016-04-01  16.550200
94  weather_hobart 2016-05-01  11.507171
95  weather_hobart 2016-06-01   8.244418


# **3 | Modeling**
---

In [13]:
# Lade das vortrainierte Chronos-2 Modell
# Chronos-2: 120M Parameter, unterstützt univariate, multivariate und covariate-basierte Prognosen
pipeline = Chronos2Pipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="cuda",
    dtype=torch.bfloat16,
)

print(f"Modell geladen: amazon/chronos-2 (Chronos2Pipeline)")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

Modell geladen: amazon/chronos-2 (Chronos2Pipeline)


# **4 | Evaluate**
---


<p><font color='black' size="5">
Exkurs:
</font></p>

**Quantile in Zeitreihenprognosen**

  Die Quantile (quantile_levels) beschreiben die Unsicherheit der Prognose durch probabilistische Vorhersagen:

  Bedeutung der einzelnen Quantile:

  | Quantil   | Bedeutung     | Interpretation                                                                        |
  |-----------|---------------|---------------------------------------------------------------------------------------|
  | 0.1 (10%) | Untere Grenze | Nur 10% der möglichen Werte liegen darunter→ Pessimistisches Szenario                 |
  | 0.5 (50%) | Median        | 50% der Werte liegen darunter, 50% darüber→ Beste Schätzung (robuster als Mittelwert) |
  | 0.9 (90%) | Obere Grenze  | 90% der möglichen Werte liegen darunter→ Optimistisches Szenario                      |

  Konfidenzintervall:

  Die Kombination von 0.1 und 0.9 ergibt ein 80% Konfidenzintervall:
  - Mit 80% Wahrscheinlichkeit liegt der tatsächliche Wert zwischen diesen beiden Grenzen
  - Je breiter das Intervall, desto unsicherer ist die Prognose

  Praktisches Beispiel:

  Prognose für Juli 2016:
  - 0.1: 8.4°C (untere Grenze)
  - 0.5: 9.4°C (beste Schätzung)
  - 0.9: 10.2°C (obere Grenze)

  → Mit 80% Wahrscheinlichkeit liegt die Temperatur zwischen 8.4°C und 10.2°C

  Warum nicht nur einen Wert?

  Zeitreihenprognosen sind unsicher. Statt nur zu sagen "es werden 9.4°C", sagt Chronos-2: "Wahrscheinlich 9.4°C, aber mit 80% Sicherheit zwischen 8.4°C und 10.2°C". Das ist wissenschaftlich ehrlicher und
  praktisch wertvoller für Entscheidungen.

<p><font color='black' size="5">
Prognose
</font></p>

In [14]:
# Prognose mit Chronos-2 erstellen
# predict_df() gibt einen DataFrame mit Prognosen und Quantilen zurück
forecast_df = pipeline.predict_df(
    context_df,
    prediction_length=prediction_length,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column="item_id",
    timestamp_column="timestamp",
    target="target"
)

print(f"Forecast DataFrame Shape: {forecast_df.shape}")
print(f"\nErste Zeilen der Prognose:")
print(forecast_df.head())
print(f"\nSpalten: {list(forecast_df.columns)}")

Forecast DataFrame Shape: (24, 7)

Erste Zeilen der Prognose:
          item_id  timestamp target_name  predictions       0.1      0.5  \
0  weather_hobart 2016-07-01      target       8.0625   6.68750   8.0625   
1  weather_hobart 2016-08-01      target       9.0000   7.65625   9.0000   
2  weather_hobart 2016-09-01      target      11.0625   9.62500  11.0625   
3  weather_hobart 2016-10-01      target      13.1250  11.68750  13.1250   
4  weather_hobart 2016-11-01      target      15.6250  14.31250  15.6250   

       0.9  
0   9.4375  
1  10.5625  
2  12.4375  
3  14.5625  
4  17.1250  

Spalten: ['item_id', 'timestamp', 'target_name', 'predictions', '0.1', '0.5', '0.9']


In [15]:
# Prognose-Struktur anzeigen
print(f"Prognose-Spalten: {list(forecast_df.columns)}")
print(f"Anzahl Prognose-Zeilen: {len(forecast_df)}")
forecast_df.head()

Prognose-Spalten: ['item_id', 'timestamp', 'target_name', 'predictions', '0.1', '0.5', '0.9']
Anzahl Prognose-Zeilen: 24


,item_id,timestamp,target_name,predictions,0.1,0.5,0.9
0,weather_hobart,2016-07-01,target,8.0625,6.68750,8.0625,9.4375
1,weather_hobart,2016-08-01,target,9.0000,7.65625,9.0000,10.5625
2,weather_hobart,2016-09-01,target,11.0625,9.62500,11.0625,12.4375
3,weather_hobart,2016-10-01,target,13.1250,11.68750,13.1250,14.5625
4,weather_hobart,2016-11-01,target,15.6250,14.31250,15.6250,17.1250


<p><font color='black' size="5">
Analysewürfel
</font></p>

In [16]:
# Prognose-Daten vorbereiten
# Chronos-2 gibt einen DataFrame zurück mit timestamp, item_id und Quantil-Spalten
# Die ersten 12 Monate der Prognose sollen mit den IST-Daten der letzten 12 Monate verglichen werden
start_date = df.index[-12]  # Startet bei den letzten 12 Monaten
forecast_index = pd.date_range(start=start_date, periods=prediction_length, freq='MS')

# Extrahiere Quantile aus dem forecast_df
# Die Spalten heißen einfach '0.1', '0.5', '0.9'
median = forecast_df['0.5'].values
low = forecast_df['0.1'].values
high = forecast_df['0.9'].values

# Historische Daten und Prognose in DataFrames umwandeln
df_forecast = pd.DataFrame({
    "index": forecast_index,
    "median": median,
    "low": low,
    "high": high
})

# Aufteilen in Vergleichsperiode (erste 12 Monate) und Zukunftsprognose (letzte 12 Monate)
df_forecast_compare = df_forecast.iloc[:12].copy()  # Erste 12 Monate für Vergleich
df_forecast_future = df_forecast.iloc[12:].copy()    # Letzte 12 Monate echte Prognose

print(f"Prognose Vergleichsperiode: {df_forecast_compare['index'].min().strftime('%Y-%m')} bis {df_forecast_compare['index'].max().strftime('%Y-%m')}")
print(f"Prognose Zukunft: {df_forecast_future['index'].min().strftime('%Y-%m')} bis {df_forecast_future['index'].max().strftime('%Y-%m')}")

Prognose Vergleichsperiode: 2016-07 bis 2017-06
Prognose Zukunft: 2017-07 bis 2018-06


In [17]:
# Vergleich IST vs. Prognose für die letzten 12 Monate
df_last_12_months = df.iloc[-12:].copy()

# Erstelle Vergleichs-DataFrame
df_comparison = pd.DataFrame({
    'YearMonth': df_last_12_months.index,
    'MedTemp_IST': df_last_12_months['MedTemp'].values,
    'MedTemp_Prognose': df_forecast_compare['median'].values,
    'low': df_forecast_compare['low'].values,
    'high': df_forecast_compare['high'].values
})

print("\nVergleich IST vs. Prognose (letzte 12 Monate):")
print(df_comparison[['YearMonth', 'MedTemp_IST', 'MedTemp_Prognose']].head())
print(f"\nAnzahl Vergleichswerte: {len(df_comparison)}")


Vergleich IST vs. Prognose (letzte 12 Monate):
   YearMonth  MedTemp_IST  MedTemp_Prognose
0 2016-07-01     9.375721            8.0625
1 2016-08-01     9.333580            9.0000
2 2016-09-01    12.977155           11.0625
3 2016-10-01    13.073045           13.1250
4 2016-11-01    15.759500           15.6250

Anzahl Vergleichswerte: 12


<p><font color='black' size="5">
Bestimmtheitsmass
</font></p>

In [18]:
# R² nur für die Vergleichsperiode (erste 12 Monate der Prognose vs. IST)
r2 = r2_score(df_comparison['MedTemp_IST'], df_comparison['MedTemp_Prognose'])
print(f"Bestimmtheitsmass (R²) für Vergleichsperiode (12 Monate): {r2:.3f}")

Bestimmtheitsmass (R²) für Vergleichsperiode (12 Monate): 0.894


<p><font color='black' size="5">
Visualisierung
</font></p>

In [19]:
def fig1():
    fig1 = go.Figure()

    # Historische Daten (nur Trainingsdaten, bis -12 Monate)
    df_train = df.iloc[:-12]
    fig1.add_trace(go.Scatter(
        x=df_train.index,
        y=df_train["MedTemp"],
        mode='lines',
        name='Historische Daten (Training)',
        line=dict(color='royalblue')
    ))

    # IST-Daten der letzten 12 Monate (zum Vergleich)
    fig1.add_trace(go.Scatter(
        x=df_last_12_months.index,
        y=df_last_12_months["MedTemp"],
        mode='lines',
        name='IST-Daten (letzte 12 Monate)',
        line=dict(color='green', width=2)
    ))

    # Prognose (alle 24 Monate)
    fig1.add_trace(go.Scatter(
        x=forecast_index,
        y=median,
        mode='lines',
        name='Median Prognose (24 Monate)',
        line=dict(color='tomato', width=2)
    ))

    # Konfidenzintervalle
    fig1.add_trace(go.Scatter(
        x=forecast_index,
        y=low,
        fill=None,
        mode='lines',
        line=dict(color='tomato', dash='dash', width=1),
        showlegend=False
    ))
    fig1.add_trace(go.Scatter(
        x=forecast_index,
        y=high,
        fill='tonexty',
        mode='lines',
        line=dict(color='tomato', dash='dash', width=1),
        showlegend=False,
        fillcolor='rgba(255, 99, 71, 0.3)',
        name='90% Konfidenzintervall'
    ))

    # Vertikale Linie zur Trennung Training/Test
    fig1.add_vline(
        x=df.index[-12].timestamp() * 1000,
        line_dash="dash",
        line_color="gray",
        annotation_text="Start Testperiode",
        annotation_position="top"
    )

    # X-Achse mit Jahr-Monat-Formatierung
    fig1.update_layout(
        title='Historische Daten und 24-Monats-Prognose',
        xaxis_title='Jahr-Monat',
        yaxis_title='Mittlere Temperatur (°C)',
        template='plotly_white',
        width=1200,
        height=500,
        xaxis=dict(
            tickformat='%Y-%m',
            dtick='M12'  # Tick alle 12 Monate
        ),
        legend=dict(
            x=0.98,  # Adjusted x-position
            y=1.32, # Adjusted y-position to move it slightly higher
            xanchor='right',
            yanchor='top',
            bordercolor='LightGray',
            borderwidth=1
        )
    )
    fig1.show()
fig1()

In [20]:
# Plot Vergleich IST vs. Prognose (12 Monate) + Zukunftsprognose (12 Monate)
def fig2():
    fig2 = go.Figure()

    # IST-Daten der letzten 12 Monate
    fig2.add_trace(go.Scatter(
        x=df_comparison['YearMonth'],
        y=df_comparison['MedTemp_IST'],
        mode='lines+markers',
        name='IST-Daten',
        line=dict(color='green', width=2),
        marker=dict(size=8, symbol='circle')
    ))

    # Prognose für Vergleichsperiode (erste 12 Monate)
    fig2.add_trace(go.Scatter(
        x=df_comparison['YearMonth'],
        y=df_comparison['MedTemp_Prognose'],
        mode='lines+markers',
        name='Prognose (Vergleichsperiode)',
        line=dict(color='orange', width=2, dash='dot'),
        marker=dict(size=8, symbol='diamond')
    ))

    # Konfidenzintervall Vergleichsperiode
    fig2.add_trace(go.Scatter(
        x=df_comparison['YearMonth'],
        y=df_comparison['low'],
        fill=None,
        mode='lines',
        line=dict(color='orange', dash='dash', width=1),
        showlegend=False
    ))
    fig2.add_trace(go.Scatter(
        x=df_comparison['YearMonth'],
        y=df_comparison['high'],
        fill='tonexty',
        mode='lines',
        line=dict(color='orange', dash='dash', width=1),
        showlegend=False,
        fillcolor='rgba(255, 165, 0, 0.2)'
    ))

    # Zukunftsprognose (letzte 12 Monate)
    df_forecast_future_indexed = df_forecast_future.set_index('index')
    fig2.add_trace(go.Scatter(
        x=df_forecast_future_indexed.index,
        y=df_forecast_future_indexed['median'],
        mode='lines+markers',
        name='Zukunftsprognose',
        line=dict(color='tomato', width=2),
        marker=dict(size=8, symbol='square')
    ))

    # Konfidenzintervall Zukunftsprognose
    fig2.add_trace(go.Scatter(
        x=df_forecast_future_indexed.index,
        y=df_forecast_future_indexed['low'],
        fill=None,
        mode='lines',
        line=dict(color='tomato', dash='dash', width=1),
        showlegend=False
    ))
    fig2.add_trace(go.Scatter(
        x=df_forecast_future_indexed.index,
        y=df_forecast_future_indexed['high'],
        fill='tonexty',
        mode='lines',
        line=dict(color='tomato', dash='dash', width=1),
        showlegend=False,
        fillcolor='rgba(255, 99, 71, 0.2)'
    ))

    # Vertikale Linie zur Trennung Vergleich/Zukunft
    fig2.add_vline(
        x=df_forecast_future_indexed.index[0].timestamp() * 1000,
        line_dash="dash",
        line_color="gray",
        annotation_text="Start Zukunftsprognose",
        annotation_position="top"
    )

    # X-Achse mit Jahr-Monat-Formatierung
    fig2.update_layout(
        title=f'Vergleich IST vs. Prognose (12 Mon.) + Zukunftsprognose (12 Mon.) | R² = {r2:.3f}',
        xaxis_title='Jahr-Monat',
        yaxis_title='Mittlere Temperatur (°C)',
        template='plotly_white',
        width=1200,
        height=500,
        xaxis=dict(
            tickformat='%Y-%m',
            tickmode='auto',
            nticks=15
        ),
        legend=dict(
            x=0.98,
            y=1.32, # Adjusted y-position to move it slightly higher
            xanchor='right',
            yanchor='top',
            bordercolor='LightGray',
            borderwidth=1
        )
    )
    fig2.show()
fig2()

# **5 | Deploy**
---